## Aligning FLYWIRE to MCNS using warpfield

### Requirements

For this tutorial we will need:
- `fw_synapse_points.feather` (49Mb): the FLYWIRE synapse cloud (30x subsampled)
- `mcns_synapse_points.feather` (64Mb): t-bars for the entire male CNS (10x subsampled)
- [`warpfield`](https://github.com/danionella/warpfield)
- `pandas`, `numpy`, `matplotlib`, `scipy`
- a runtime with a GPU

If you're in Google Colab, make sure your runtime is set to use a GPU:
- Go to `Runtime` -> `Change runtime type`
- Select `T4 GPU` under `Hardware accelerator` 

In [ ]:
%%capture
!wget https://flyem.mrc-lmb.cam.ac.uk/flyconnectome/imagereg_workshop/mcns_synapse_points.feather
!wget https://flyem.mrc-lmb.cam.ac.uk/flyconnectome/imagereg_workshop/fw_synapse_points.feather

In [ ]:
%%capture
!pip install pandas numpy matplotlib scipy warpfield

In [ ]:
import warpfield

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from scipy import ndimage

First, we will generate our synapse density images for registration. 

In [ ]:
# Load the male CNS t-bars data (this is 8x8x8nm)
points_mcns = pd.read_feather("mcns_synapse_points.feather")

# Convert to microns
points_mcns = points_mcns[["x", "y", "z"]].values / 125

# Convert to 1x1x1um resolution pixels
points_mcns = (points_mcns // 1).astype(int)

# Create a 3D histogram
mx = points_mcns.max(axis=0) + 1
img_mcns = np.histogramdd(
    points_mcns, bins=[np.arange(mx[0] + 1), np.arange(mx[1] + 1), np.arange(mx[2] + 1)]
)[0]

# Trim image to just the brain and remove some ventral space
img_mcns_brain = img_mcns[:, :450, :400]

# Smooth
img_mcns_brain_smooth = ndimage.gaussian_filter(img_mcns_brain, sigma=1)

# Normalize (note that we normalize to 99th percentile to avoid outliers)
img_mcns_brain_smooth = np.clip(
    img_mcns_brain_smooth / np.percentile(img_mcns_brain_smooth, 99) * 255, 0, 255
).astype(np.float32)

plt.imshow(img_mcns_brain_smooth.sum(axis=2).T, cmap="gray")

del points_mcns
del img_mcns
del img_mcns_brain

In [ ]:
# This data is in nm resolution
points_fw = pd.read_feather(
    "fw_synapse_points.feather"
).values

# # Convert to 1x1x1um resolution voxels
points_fw = (points_fw // 1000).astype(int)

# # Create a 3D histogram
mx = points_fw.max(axis=0) + 1
img_fw = np.histogramdd(
    points_fw, bins=[np.arange(mx[0] + 1), np.arange(mx[1] + 1), np.arange(mx[2] + 1)]
)[0]

# Smooth
img_fw_smooth = ndimage.gaussian_filter(img_fw, sigma=1)

# Normalize (note that we normalize to 99th percentile to avoid outliers)
img_fw_smooth = np.clip(
    img_fw_smooth / np.percentile(img_fw_smooth, 99) * 255, 0, 255
).astype(np.float32)

# Plot frontal view
plt.imshow(img_fw_smooth.sum(axis=2).T, cmap="gray")

del points_fw
del img_fw

In [ ]:
# Load the default recipe (this includes affine + non-rigid registration)
recipe = warpfield.Recipe.from_yaml('default.yml')

# Define fixed and moving images
fixed, moving = img_mcns_brain_smooth, img_fw_smooth

In [ ]:
# Run the default registration
vol_mov_reg, warp_map, _ = warpfield.register_volumes(fixed, moving, recipe)

In [ ]:
# `vol_mov_reg` is the FlyWire image in male CNS space
plt.imshow(vol_mov_reg.sum(axis=2).T, cmap="gray")

In [ ]:
# Let's illustrate the registration process with a video
video_path = "output.mp4"
units_per_voxel = [1,1,1]
callback = warpfield.utils.mips_callback(units_per_voxel=units_per_voxel)
moving_reg, warpmap, _ = warpfield.register_volumes(fixed, moving, recipe, video_path=video_path, callback=callback)
warpfield.utils.showvid(video_path, embed=True)

In [ ]:
# Same but with slices
video_path = "output.mp4"
units_per_voxel = [1, 1, 1]
callback = warpfield.utils.mosaic_callback(units_per_voxel=units_per_voxel, axis=0, num_slices=9)
moving_reg, warpmap, _ = warpfield.register_volumes(fixed, moving, recipe, video_path=video_path, callback=callback)
warpfield.utils.showvid(video_path, embed=True)

In [ ]:
# You can use the warp map to push/pull coordinates from one space to another
help(warpmap.push_coordinates)

In [ ]:
# There are option to export the warp map to other formats
help(warpmap.to_h5)